In [17]:
import torch


# GPU 할당 변경하기
GPU_NUM = 5  # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')

torch.cuda.set_device(device) # change allocation of current GPU
print ('Current cuda device ', torch.cuda.current_device()) # check

Current cuda device  5


In [27]:
import numpy as np
from sklearn.preprocessing import StandardScaler

In [18]:
S_left  = np.load('S_left.npy')
S_right = np.load('S_right.npy')
S_left_phase  = np.load('S_left_phase.npy')
S_right_phase = np.load('S_right_phase.npy')

In [19]:
S_left  = torch.as_tensor(S_left ).to('cuda:5')
S_right = torch.as_tensor(S_right).to('cuda:5')
S_left_phase  = torch.as_tensor(S_left_phase ).to('cuda:5')
S_right_phase = torch.as_tensor(S_right_phase).to('cuda:5')

In [20]:
x_data = torch.empty(1,1,257,382)
y_data = torch.as_tensor(np.load('label.npy')).to('cuda:5')

In [24]:
def dB( magnitude ):

    eps = torch.finfo(float).eps
    decibel = 20*torch.log10( torch.abs(magnitude) + eps )
    return decibel




def scaler( L, R ):
    
    LR = torch.cat( [L,R], dim=0 )
    LR_cpu = LR.to('cpu')
    LR_npy = np.asarray(LR_cpu)
    
    z = StandardScaler().fit_transform(LR_npy[:])
    z = torch.as_tensor(z).to('cuda')
    z = z.reshape(2, 257, 382)

    return z[0], z[1]




def angle_to_digit( y_data ):
    
    for idx in range(y_data.shape[0]):
        
        y = y_data[idx]
        if y<0: y_data[idx] = 10
        else:   y_data[idx] = ( y//20 )
    
    return y_data


In [22]:
S_left.shape

torch.Size([257, 382, 500])

In [28]:

x_data = torch.empty([1,4,257,382]).to(device)


for idx in range( y_data.shape[0] ):
    
    
    mag_L = S_left[:,:,idx]
    mag_R = S_right[:,:,idx]
    phi_L = S_left_phase[:,:,idx]
    phi_R = S_right_phase[:,:,idx]


    """log[dB] scale"""
    log_L = dB( mag_L )
    log_R = dB( mag_R )


    """정규화"""
    #z_mag = scaler( mag_L, mag_R )
    z_mag = scaler( log_L, log_R )
    z_phi = scaler( phi_L, phi_R )
    
    
    x_left_mag ,  x_right_mag   =  z_mag
    x_left_phase, x_right_phase =  z_phi
    
    
    xl_m = x_left_mag.reshape(1,257,382)
    xl_p = x_left_phase.reshape(1,257,382)
    xr_m = x_right_mag.reshape(1,257,382)
    xr_p = x_right_phase.reshape(1,257,382)
    
    x_element = torch.cat([ xl_m, xr_m, xl_p, xr_p ], dim=0)
    x_element = x_element.reshape(1, 4, 257, 382)
    #(1, 4, 257, 382)
 
    
    x_data = torch.cat([x_data, x_element], dim=0)

x_data = x_data[1:]
y_data = torch.as_tensor( angle_to_digit(y_data) ).reshape(500,)


    

In [29]:
torch.save( x_data, 'validation_dataset/x_data.pt' )
torch.save( y_data, 'validation_dataset/y_data.pt')